## Sentiment Analysis with WordVec

There's two approaches to sentiment analysis: using a pretrained lexicon to look up the sentiment of each individual word to generate an composite score, and the machine learning approach tends to be more accurate. This is because lexicons don't handle context, making them poor at handling subtleties in language (like metaphors, dual meanings and sarcasm).

Deep neural are better at understanding these subtleties because they don't analyze text at face value. Instead, they create abstract representations of what they've learned. These generalizations are called vectors and we can use them to classify data. This notebook demonstrates how to use vectors to build a sentiment classifier for movie reviews, based on this [sample code](https://github.com/tflearn/tflearn/blob/master/examples/nlp/lstm.py). If you're interested in an explanation of how it works, look at [this video by Siraj Raval](https://www.youtube.com/watch?v=si8zZHkufRY).

We start with the necessary imports, install [TFLearn](http://tflearn.org) if you haven't already.

In [16]:
from __future__ import division, print_function, absolute_import

import tflearn
import numpy as np
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb

The first step is to load our data set, which is helpfully bundled with tflearn as a pre-processed data sets. 

The dataset is of 25000 movie reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words". As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

The load_data function downloads our data set from the web in a pkl (pickle) byte stream. We want 10,000 words from the database, and we only want to use 10% of the data for our validation set. While training data helps fit our weights, validation data is used during training to help tune the hyper-parameters.

The load_data function will return our movie review split into a training and testing set, which we can then further split those sets into reviews (trainX & testX) and labels (trainY & testY).


In [2]:
# IMDB Dataset loading
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000, valid_portion=0.1)
trainX, trainY = train
testX, testY = test

We can't just feed text strings into a neural network directly, we have to vectorise our inputs. The pad_sequences function will do this for our review text - it'll convert each review into a matrix and pad it.

Padding is necessary to ensure consistency in our inputs' dimensionality. It will pad each sequence with a zero at the end which we specify until it reaches the maximum possible sequence length (which we set to 100).

We also want to convert our labels to vectors as well and we do that using the to_categorical function to create  binary vectors with two classes, 1 (which is positive) or 0 (which is negative).

In [3]:
# Data preprocessing
# Sequence padding
trainX = pad_sequences(trainX, maxlen=100, value=0.)
testX = pad_sequences(testX, maxlen=100, value=0.)
# Converting labels to binary vectors
trainY = to_categorical(trainY, nb_classes=2)
testY = to_categorical(testY, nb_classes=2)

Previously, we've used a feed-forward net for sentiment analysis, but FFNs have limitations - their input size is fixed, and they can't handle sequences. So this example will use a Recurrent Neural Network (RNN), which can take  word embedding vectors of any size as input.

The RNN consists of a stack of layers, the first being the input layer, where we feed data into our network. The only parameter needed is the input shape, a tuple consisting of the batch size, (set to none) and the input length, which is set to 100, in line with the max sequence length we set earlier.

Next is our embedding layer. The first parameter is the output vector we receive from the previous layer, (which is standard practice, this is how data flows through a neural network). We set the dimensions to 10,000 since that's how many words we loaded from our data set earlier, and the output dimension to 128, which is the number of dimensions in our resulting embeddings. Notice the dimensionality reduction embeddings provide.

Next, we'll feed those values to our Long Short-Term Memory (LSTM) layer. This layer allows our network to remember data from the beginning of the sequences, which will improve our prediction.

We set dropout to 0.8 which is a technique that helps prevent overfitting by randomly turning on and off different pathways in our network (an 80% chance of a path being muted during each batch process).

Our next layer is fully connected which means that every neuron in the previous layer is connected to every neuron in this layer. By now we will have a set of learned feature vectors from previous layers, and adding a fully connected layer is a computationally cheap way of learning non-linear combinations of them. It consists of two units, and it's using softmax  as its activation function. This will take in a vector of values and squash it into a vector of output probabilities between 0 and 1, that sum to 1.

We use those values in our last layer, which is our regression layer. This will apply a regression operation to the input. We're going to specify an optimizer method that will minimize a given loss function, as well as the learning rate, which specifies how fast we want our network to train. The optimizer we'll use is adam, which performs gradient descent. And categorical cross entropy is our loss, it helps to find the difference between our predicted output and the expected output.

In [4]:
# Network building
net = tflearn.input_data([None, 100])
net = tflearn.embedding(net, input_dim=10000, output_dim=128)
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy')

After building our neural network, we can go ahead and initialize it using tflearn's deep neural net function. Then we can call our model's fit function, which will launch the training process for our given training and validation data. We'll also set show metric to true so we can view the log of accuracy during training.

In [6]:
# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True, batch_size=32, n_epoch=3)

Training Step: 2111  | time: 178.521s
| Adam | epoch: 003 | loss: 0.00000 - acc: 0.0000 -- iter: 22496/22500
Training Step: 2112  | time: 183.450s
| Adam | epoch: 003 | loss: 0.00000 - acc: 0.0000 | val_loss: 0.46445 - val_acc: 0.8172 -- iter: 22500/22500
--


Finally, let's try using our model to generate predictions

In [31]:
# Find the indices of the most confident prediction for each item.
predictions = np.array(model.predict(testX)).argmax(axis=1)

# Calculate the accuracy, which is the percentage of times the predicated labels matched the actual labels
actual = testY.argmax(axis=1)
test_accuracy = np.mean(predictions == actual, axis=0)

# Print out the result
print("Test accuracy: ", test_accuracy)

Test accuracy:  0.8172


Because there's no reverse index supplied with this data set, we can't create our own word vectors from new text. So we can only run predictions with features from our existing test set.  

In [41]:
# reformat to 2d, as predict expects rows of features
x = testX[0][None,:]
print (x.shape)

# show the predicted probabilities
prediction = np.array(model.predict(x))
print ("Prediction probs: ", prediction)

# show the predicted label (as a one-hot encoding)
prediction = model.predict_label(x)
print ("Predicted", prediction, " actually:", testY[0])


(1, 100)
Prediction probs:  [[ 0.74907649  0.25092345]]
Predicted [[1 0]]  actually: [ 1.  0.]
